In [1]:
import pandas as pd
import numpy as np
import csv
import os
import matplotlib.pyplot as plt
import warnings
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn import linear_model
from sklearn.metrics import accuracy_score, f1_score, ConfusionMatrixDisplay, confusion_matrix
from datetime import date
import bisect
from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.inspection import DecisionBoundaryDisplay
pd.options.mode.chained_assignment = None
warnings.simplefilter(action='ignore', category=FutureWarning)
DATA_FILE = 'pyspark_data.csv'

In [2]:
pre_data = pd.read_csv(DATA_FILE)
pre_data

,team,score,walksissued,stolenbases,strikeoutsthrown,totalbases,date,opponent,opponent_score,home
0,STL,1,5,0,5,5,2016-04-03,PIT,4,0
1,TOR,5,1,0,7,11,2016-04-03,TB,3,0
2,NYM,3,2,0,3,8,2016-04-04,KC,4,0
3,SF,12,5,1,10,30,2016-04-04,MIL,3,0
4,MIN,2,5,0,10,10,2016-04-04,BAL,3,0
...,...,...,...,...,...,...,...,...,...,...
26853,HOU,7,2,2,12,14,2021-10-28,ATL,2,1
26854,ATL,2,3,0,7,11,2021-10-30,HOU,0,1
26855,ATL,3,5,0,7,15,2021-10-31,HOU,2,1
26856,ATL,5,6,0,9,15,2021-11-01,HOU,9,1


In [3]:
game_data = list()
mlb_teams = ["PIT", "TB", "KC", 'MIL', 'BAL', 'TEX', 'ATL', 'CIN', 'SD', 'ARI', 'LAA', 'OAK', 'NYY', 'SEA', 'WSH', \
             'TOR', 'NYM', 'SF', 'MIN', 'PHI', 'HOU', 'CHW', 'BOS', 'STL', 'DET', 'LAD', 'COL', 'CHC', 'CLE', 'MIA']
for team in mlb_teams:
    t = pre_data[pre_data['team'] == team]
    t['date'] = pd.to_datetime(t['date'])
    t.drop_duplicates(subset=['date'])
    t.reset_index()
    t.sort_values(by='date',inplace=True)
    print("Number of sample for team " + team + ": ", len(t))
    game_data.append(t)


Number of sample for team PIT:  877
Number of sample for team TB:  914
Number of sample for team KC:  879
Number of sample for team MIL:  901
Number of sample for team BAL:  881
Number of sample for team TEX:  883
Number of sample for team ATL:  915
Number of sample for team CIN:  882
Number of sample for team SD:  886
Number of sample for team ARI:  884
Number of sample for team LAA:  884
Number of sample for team OAK:  885
Number of sample for team NYY:  915
Number of sample for team SEA:  879
Number of sample for team WSH:  909
Number of sample for team TOR:  892
Number of sample for team NYM:  882
Number of sample for team SF:  891
Number of sample for team MIN:  882
Number of sample for team PHI:  878
Number of sample for team HOU:  953
Number of sample for team CHW:  889
Number of sample for team BOS:  909
Number of sample for team STL:  891
Number of sample for team DET:  876
Number of sample for team LAD:  957
Number of sample for team COL:  883
Number of sample for team CHC:  

In [4]:
pred_data = dict()
actual_data = dict()
for td in game_data:
    td['score_10'] = td['score'].shift(1).rolling(window=10).mean()
    td['walksissued_10'] = td['walksissued'].shift(1).rolling(window=10).mean()
    td['stolenbases_10'] = td['stolenbases'].shift(1).rolling(window=10).mean()
    td['strikeoutsthrown_10'] = td['strikeoutsthrown'].shift(1).rolling(window=10).mean()
    td['totalbases_10'] = td['totalbases'].shift(1).rolling(window=10).mean()
    td['win'] = 1
    td.loc[td['score'] < td['opponent_score'], 'win'] = 0
    td = td.dropna()
    X = td[['score_10','walksissued_10', 'stolenbases_10', 'strikeoutsthrown_10', 'totalbases_10', 'home']]
    y = td['score']
    yo = td['opponent_score']
    tmp = td['date'].tolist()
#     t = bisect.bisect(tmp, date(2021, 4, 1))
    t = int(len(td['score']) * 0.7)
    X_train, y_train = X[:t], y[:t]
    X_test, y_test, yo_test = X[t:], y[t:].tolist(), yo[t:].tolist()
    X_test_date = td['date'][t:].tolist()
    X_test_opponent = td['opponent'][t:].tolist()
    y_win = td['win'][t:].tolist()
    
    print(X_train, y_train)
#     clf = RandomForestClassifier(max_depth=2, random_state=0)
#     clf = RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
#     clf = KNeighborsClassifier(10)
    clf = SVC(kernel="linear", C=0.025)
#     SVC(gamma=2, C=1)
#     clf = GaussianProcessClassifier(1.0 * RBF(1.0))
#     clf =  DecisionTreeClassifier(max_depth=5)
#     RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1)
#     clf = MLPClassifier(alpha=1, max_iter=1000)
#     clf =  AdaBoostClassifier()
#     clf =  GaussianNB()
#     clf = QuadraticDiscriminantAnalysis()
#     clf = AdaBoostClassifier(n_estimators=400)
#     clf = linear_model.LinearRegression()
#     clf = linear_model.LassoLars(alpha=.1)
#     clf = linear_model.LogisticRegression()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    pred_data[td['team'].iloc[0]] = {'date': X_test_date, 'pred_score': y_pred, 'opponent': X_test_opponent, 'win': y_win, 'opponent_score': yo_test, 'score': y_test}
    
#     acc = accuracy_score(y_win, y_pred_win)
#     f1 = f1_score(y_win, y_pred_win, average='macro')
#     print("=====  Prediction for team " + td['team'].iloc[0] + " =====")
#     print("    Accuracy: ", acc)
#     print("    F1 score: ", f1)
#     predicted_score = pd.DataFrame(predicted_score, index=y_test.index, columns = ['Predicted Score'])
#     predicted_score['Actual Price'] = y_test
#     predicted_score['Error'] = (predicted_score['Predicted Price'] - y_test)/y_test
#     predicted_score.insert(loc = 0, column = 'Date', value = price['date'].loc[predicted_score.index[0]:])
#     predicted_score.insert(loc = 1, column = 'Stock', value = price['team'].loc[price['team'].index[0]])
    

       score_10  walksissued_10  stolenbases_10  strikeoutsthrown_10  \
13569       3.9             4.0             0.4                  8.2   
13590       3.9             4.5             0.7                  7.6   
13602       3.8             4.7             0.8                  6.9   
202         4.2             4.9             0.8                  6.8   
217         4.0             5.3             1.0                  7.2   
...         ...             ...             ...                  ...   
9329        5.0             2.7             0.8                  9.0   
9344        5.2             3.1             0.7                  9.0   
22787       5.1             3.0             0.6                  9.4   
22801       5.2             3.0             0.5                  9.3   
22829       4.5             2.9             0.5                  9.4   

       totalbases_10  home  
13569           13.2     1  
13590           13.2     1  
13602           13.2     1  
202             14.

       score_10  walksissued_10  stolenbases_10  strikeoutsthrown_10  \
154         4.0             4.8             0.6                  7.4   
176         4.3             5.3             0.6                  6.9   
13612       4.3             5.4             0.6                  6.2   
13625       3.4             5.3             0.5                  6.8   
13633       3.3             4.9             0.7                  6.3   
...         ...             ...             ...                  ...   
22805       5.1             3.6             0.2                  9.3   
22820       4.9             3.6             0.2                  9.2   
22833       4.8             3.4             0.2                  9.9   
22845       4.5             3.1             0.2                 10.0   
22855       3.8             3.2             0.5                 10.3   

       totalbases_10  home  
154             12.2     0  
176             14.0     0  
13612           13.9     1  
13625           12.

      score_10  walksissued_10  stolenbases_10  strikeoutsthrown_10  \
152        3.9             3.0             0.2                 10.1   
167        4.0             3.0             0.3                  9.8   
190        3.3             2.7             0.3                  9.1   
207        2.6             3.0             0.4                  8.4   
219        2.6             3.0             0.4                  8.6   
...        ...             ...             ...                  ...   
9321       3.2             2.2             1.0                  7.1   
9341       3.5             2.5             0.7                  7.9   
9346       3.0             2.4             0.5                  7.7   
9360       3.5             2.0             0.8                  8.2   
9373       3.8             1.9             0.7                  8.1   

      totalbases_10  home  
152            13.3     0  
167            13.9     0  
190            12.1     0  
207            10.9     0  
219    

       score_10  walksissued_10  stolenbases_10  strikeoutsthrown_10  \
13591       4.2             3.2             0.9                  8.1   
13576       4.0             2.9             0.7                  8.5   
13604       3.5             2.3             0.8                  8.2   
199         3.5             2.3             0.8                  8.6   
215         3.6             2.2             0.9                  7.6   
...         ...             ...             ...                  ...   
23109       5.4             3.3             0.4                 12.2   
23119       6.7             3.1             0.4                 12.8   
23132       7.4             3.0             0.4                 11.9   
23147       7.4             2.7             0.4                 11.6   
23165       7.3             2.6             0.4                 12.5   

       totalbases_10  home  
13591           15.0     1  
13576           15.3     1  
13604           13.4     1  
199             13.

In [5]:
pred_data

{'PIT': {'date': [Timestamp('2019-08-19 00:00:00'),
   Timestamp('2019-08-20 00:00:00'),
   Timestamp('2019-08-21 00:00:00'),
   Timestamp('2019-08-22 00:00:00'),
   Timestamp('2019-08-23 00:00:00'),
   Timestamp('2019-08-24 00:00:00'),
   Timestamp('2019-08-25 00:00:00'),
   Timestamp('2019-08-26 00:00:00'),
   Timestamp('2019-08-27 00:00:00'),
   Timestamp('2019-08-28 00:00:00'),
   Timestamp('2019-08-30 00:00:00'),
   Timestamp('2019-08-31 00:00:00'),
   Timestamp('2019-09-01 00:00:00'),
   Timestamp('2019-09-01 00:00:00'),
   Timestamp('2019-09-03 00:00:00'),
   Timestamp('2019-09-04 00:00:00'),
   Timestamp('2019-09-05 00:00:00'),
   Timestamp('2019-09-06 00:00:00'),
   Timestamp('2019-09-07 00:00:00'),
   Timestamp('2019-09-08 00:00:00'),
   Timestamp('2019-09-10 00:00:00'),
   Timestamp('2019-09-11 00:00:00'),
   Timestamp('2019-09-12 00:00:00'),
   Timestamp('2019-09-12 00:00:00'),
   Timestamp('2019-09-13 00:00:00'),
   Timestamp('2019-09-14 00:00:00'),
   Timestamp('2019-09-1

In [6]:
prediction = list()
groundtruth = list()
remove_duplicate = list()
csv_dict = list()
for team in pred_data:
    for i in range(len(pred_data[team]['date'])):
        op_team = pred_data[team]['opponent'][i]
        sc = pred_data[team]['pred_score'][i]
        day = pred_data[team]['date'][i]
        s1 = team + '-' + op_team + '-' + day.strftime("%m/%d/%Y, %H:%M:%S")
        s2 = op_team + '-' + team + '-' + day.strftime("%m/%d/%Y, %H:%M:%S")
        if s1 in remove_duplicate or s2 in remove_duplicate:
            print("duplicate match prediction")
            continue
        if op_team not in pred_data:
            print("Assert: no such team: ", op_team)
            continue
        if day not in pred_data[op_team]['date']:
            print("the match is not considered")
            continue
        op_id = pred_data[op_team]['date'].index(day)
        if pred_data[op_team]['opponent'][op_id] != team:
            print("something wrong with the match")
            continue
        remove_duplicate.append(s1)
        remove_duplicate.append(s2)
        op_sc = pred_data[op_team]['pred_score'][op_id]
        if sc >= op_sc:
            prediction.append(1)
        else:
            prediction.append(0)
        groundtruth.append(pred_data[team]['win'][i])
        csv_dict.append({'team1': team, 'team2': op_team, 'team1_pred_score':sc, 'team2_pred_score':op_sc, 'team1_act_score':pred_data[team]['score'][i], 'team2_act_score':pred_data[team]['opponent_score'][i], 'date': day.strftime("%Y/%m/%d")})
        
        
            
    

the match is not considered
the match is not considered
the match is not considered
the match is not considered
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
something wrong with the match
something wrong with the match
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
something wrong with the match
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate ma

duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
duplicate match prediction
d

In [7]:
print(len(groundtruth))

3330


In [8]:

acc = accuracy_score(groundtruth, prediction)
f1 = f1_score(groundtruth, prediction, average='macro')
print("    Accuracy: ", acc)
print("    F1 score: ", f1)

    Accuracy:  0.5162162162162162
    F1 score:  0.5156584912186395


In [9]:
# csv_file = "classification.csv"
# csv_columns = ['team1', 'team2', 'team1_pred_score', 'team2_pred_score', 'team1_act_score', 'team2_act_score', 'date']
# try:
#     with open(csv_file, 'w') as csvfile:
#         writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
#         writer.writeheader()
#         for data in csv_dict:
#             writer.writerow(data)
# except IOError:
#     print("I/O error")